In [ ]:
!python -m pip install transformers torchmetrics

In [ ]:
# !python -m pip install torchmetrics

In [ ]:
import torchmetrics

In [ ]:
import transformers

In [ ]:
import traceback
import csv

import pandas as pd


def write_tsv_dataframe(filepath, dataframe):
    """
        Stores `DataFrame` as tsv file

        Parameters
        ----------
        filepath : str
            Path to tsv file
        dataframe : pd.DataFrame
            DataFrame to store

        Raises
        ------
        IOError
            if the file can't be opened
    """
    try:
        dataframe.to_csv(filepath, encoding='utf-8', sep='\t', index=False, header=True, quoting=csv.QUOTE_NONE)
    except IOError:
        traceback.print_exc()


In [ ]:
def combine_columns(df_arguments, df_labels):
    """Combines the two `DataFrames` on column `Argument ID`"""
    return pd.merge(df_arguments, df_labels, on='Argument ID')


In [ ]:
def split_arguments(df_arguments):
    """Splits `DataFrame` by column `Usage` into `train`-, `validation`-, and `test`-arguments"""
    train_arguments = df_arguments.loc[df_arguments['Usage'] == 'train'].drop(['Usage'], axis=1).reset_index(drop=True)
    valid_arguments = df_arguments.loc[df_arguments['Usage'] == 'validation'].drop(['Usage'], axis=1).reset_index(drop=True)
    test_arguments = df_arguments.loc[df_arguments['Usage'] == 'test'].drop(['Usage'], axis=1).reset_index(drop=True)
    
    return train_arguments, valid_arguments, test_arguments


In [ ]:
def create_dataframe_head(argument_ids, model_name):
    """
        Creates `DataFrame` usable to append predictions to it

        Parameters
        ----------
        argument_ids : list[str]
            First column of the resulting DataFrame
        model_name : str
            Second column of DataFrame will contain the given model name

        Returns
        -------
        pd.DataFrame
            prepared DataFrame
    """
    df_model_head = pd.DataFrame(argument_ids, columns=['Argument ID'])
    df_model_head['Method'] = [model_name] * len(argument_ids)

    return df_model_head


In [ ]:
import json
class MissingColumnError(AttributeError):
    """Error indicating that an imported DataFrame lacks necessary columns"""
    pass


In [ ]:
def load_json_file(filepath):
    """Load content of json-file from `filepath`"""
    with open(filepath, 'r') as  json_file:
        return json.load(json_file)


In [ ]:
def load_values_from_json(filepath):
    """Load values per level from json-file from `filepath`"""
    json_values = load_json_file(filepath)
    values = { "1":set(), "2":set(), "3":set(), "4a":set(), "4b":set() }
    for value in json_values["values"]:
        values["1"].add(value["name"])
        values["2"].add(value["level2"])
        for valueLevel3 in value["level3"]:
            values["3"].add(valueLevel3)
        for valueLevel4a in value["level4a"]:
            values["4a"].add(valueLevel4a)
        for valueLevel4b in value["level4b"]:
            values["4b"].add(valueLevel4b)
    values["1"] = sorted(values["1"])
    values["2"] = sorted(values["2"])
    values["3"] = sorted(values["3"])
    values["4a"] = sorted(values["4a"])
    values["4b"] = sorted(values["4b"])
    return values


In [ ]:
def load_arguments_from_tsv(filepath, default_usage='test'):
    """
        Reads arguments from tsv file

        Parameters
        ----------
        filepath : str
            The path to the tsv file
        default_usage : str, optional
            The default value if the column "Usage" is missing

        Returns
        -------
        pd.DataFrame
            the DataFrame with all arguments

        Raises
        ------
        MissingColumnError
            if the required columns "Argument ID" or "Premise" are missing in the read data
        IOError
            if the file can't be read
        """
    try:
        dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        if not {'Argument ID', 'Premise'}.issubset(set(dataframe.columns.values)):
            raise MissingColumnError('The argument "%s" file does not contain the minimum required columns [Argument ID, Premise].' % filepath)
        if 'Usage' not in dataframe.columns.values:
            dataframe['Usage'] = [default_usage] * len(dataframe)
        return dataframe
    except IOError:
        traceback.print_exc()
        raise


In [ ]:
def load_labels_from_tsv(filepath, label_order):
    """
        Reads label annotations from tsv file

        Parameters
        ----------
        filepath : str
            The path to the tsv file
        label_order : list[str]
            The listing and order of the labels to use from the read data

        Returns
        -------
        pd.DataFrame
            the DataFrame with the annotations

        Raises
        ------
        MissingColumnError
            if the required columns "Argument ID" or names from `label_order` are missing in the read data
        IOError
            if the file can't be read
        """
    try:
        label_order = ["Self-direction: thought",	"Self-direction: action",	"Stimulation",	"Hedonism",	"Achievement",	"Power: dominance",	"Power: resources",	"Face",	"Security: personal",	"Security: societal",	"Tradition",	"Conformity: rules",	"Conformity: interpersonal",	"Humility",	"Benevolence: caring",	"Benevolence: dependability",	"Universalism: concern",	"Universalism: nature",	"Universalism: tolerance",	"Universalism: objectivity"]
        dataframe = pd.read_csv(filepath, encoding='utf-8', sep='\t', header=0)
        dataframe = dataframe[['Argument ID'] + label_order]
        return dataframe
    except IOError:
        traceback.print_exc()
        raise
    except KeyError:
        raise MissingColumnError('The file "%s" does not contain the required columns for its level.' % filepath)


In [ ]:
import sys
import getopt
import os

In [ ]:
model_dir = 'models'
data_dir = 'data'

In [ ]:
if not os.path.exists(model_dir):
    os.makedirs(model_dir)


In [ ]:
from typing import Dict, List

In [ ]:
def generate_pairwise_input(dataset, labels, datatype):
    """
    TODO: group all premises and corresponding hypotheses and labels of the datapoints
    a datapoint as seen earlier is a dict of premis, hypothesis and label
    """
    #raise NotImplementedError
    premise=[]
    conclusion=[]
    stance=[]
    n_labels =labels.keys()
    n_labels = n_labels[1:]
    print(n_labels)
    label=[]
    
    n = len(dataset['Argument ID'])
    m = len(labels['Argument ID'])
    arguments = []
    print(n,m)
    for i in range(n):
        if dataset['Usage'][i]==datatype:
          premise.append(dataset['Premise'][i])
          conclusion.append(dataset['Conclusion'][i])
          stance.append(dataset['Stance'][i])
          arguments.append(dataset['Argument ID'][i])
    for i in range(m):
        if (labels['Argument ID'][i] in arguments):
          sent_label = []
          #print(i)
          for l in range(len(n_labels)):
              #print(n_labels[l])
              sent_label.append(int(labels[n_labels[l]][i]))
          label.append(sent_label)

    return premise, conclusion, stance, label

In [ ]:
value_json_filepath = os.path.join(data_dir, 'values.json')



In [ ]:
values = load_values_from_json(value_json_filepath)
num_labels_Lv2 = len(values['2'])
level =2


In [ ]:
argument_filepath = os.path.join(data_dir, 'arguments-training.tsv')

df_arguments = load_arguments_from_tsv(argument_filepath, default_usage='train')
label_filepath = os.path.join(data_dir, 'labels-training.tsv')
df_labels = load_labels_from_tsv(label_filepath, values[str(level)])

In [ ]:
train_premises, train_conclusion, train_stance, train_labels = generate_pairwise_input(df_arguments, df_labels, 'train')


In [ ]:
len(train_labels)

In [ ]:
val_argument_filepath = os.path.join(data_dir, 'arguments-validation.tsv')
df_arguments_val = load_arguments_from_tsv(val_argument_filepath, default_usage='validation')
val_label_filepath = os.path.join(data_dir, 'labels-validation.tsv')
df_labels_val = load_labels_from_tsv(val_label_filepath, values[str(level)])


In [ ]:
val_premises, val_conclusion, val_stance, val_labels = generate_pairwise_input(df_arguments_val, df_labels_val, 'validation')

In [ ]:
# Nothing to do for this class!
import torch
from transformers import BertModel
from transformers import AutoTokenizer
from typing import Dict, List

class BatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
        """
        self.hf_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    
    def get_sep_token(self,):
        return self.hf_tokenizer.sep_token
    
    def __call__(self, prem_batch: List[str], hyp_batch: List[str], stance_batch: List[str]) -> List[List[str]]:
        """Uses the huggingface tokenizer to tokenize and pad a batch.

        We return a dictionary of tensors per the huggingface model specification.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            Dict: The dictionary of token specifications provided by HuggingFace
        """
        # The HF tokenizer will PAD for us, and additionally combine 
        # The two sentences deimited by the [SEP] token.
        batch_len = len(prem_batch)
        #spaces = [" "]*batch_len
        conc_batch = [stance_batch[i]+" "+hyp_batch[i] for i in range(batch_len)]
        enc = self.hf_tokenizer(
            prem_batch,
            conc_batch,
            padding=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )

        return enc
    

# HERE IS AN EXAMPLE OF HOW TO USE THE BATCH TOKENIZER
tokenizer = BatchTokenizer()
a = [["this is the premise.", "This is also a premise"], ["this is the hypothesis", "This is a second hypothesis"],["in favour of", "against"]]
x = tokenizer(*a)
print(x)
tokenizer.hf_tokenizer.batch_decode(x["input_ids"])



In [ ]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[:][i:i + n]

def chunk_multi(lst1, lst2, lst3, n):
    for i in range(0, len(lst1), n):
        yield lst1[i: i + n], lst2[i: i + n], lst3[i: i + n]
        


In [ ]:
# Notice that since we use huggingface, we tokenize and
# encode in all at once!
batch_size=64
tokenizer = BatchTokenizer()
train_input_batches = [b for b in chunk_multi(train_premises, train_conclusion, train_stance, batch_size)]
# Tokenize + encode
train_input_batches = [tokenizer(*batch) for batch in train_input_batches]

In [ ]:
val_input_batches = [b for b in chunk_multi(val_premises, val_conclusion, val_stance, batch_size)]
# Tokenize + encode
val_input_batches = [tokenizer(*batch) for batch in val_input_batches]


In [ ]:
len(val_labels)

In [ ]:
# 0 Self-direction: thought
# 1 Self-direction: action	
# 2 Stimulation	
# 3 Hedonism	
# 4 Achievement	
# 5 Power: dominance	
# 6 Power: resources	
# 7 Face	
# 8 Security: personal	
# 9 Security: societal	
# 10 Tradition	
# 11 Conformity: rules	
# 12 Conformity: interpersonal	
# 13 Humility	
# 14 Benevolence: caring	
# 15 Benevolence: dependability	
# 16 Universalism: concern	
# 17 Universalism: nature	
# 18 Universalism: tolerance	
# 19 Universalism: objectivity

In [ ]:
# Achievement, Face, Power: dominance, Power: resources [4, 5, 6, 7]


#Benevolence: caring, Benevolence: dependability, Humility, Universalism: concern [13,14,15, 16]


# Stimulation, Tradition, Self-direction: action, Self-direction: thought [2, 10, 0, 1]


# Conformity: interpersonal, Conformity: rules, Security: personal, Security: societal [11, 12, 8, 9]


# Hedonism, Universalism: nature, Universalism: objectivity, Universalism: tolerance [3, 17,18, 19]
# Note: This is just one possible way to group these elements. There may be other valid ways to do so.


clusters = [
    [4, 5, 6, 7],
    [13,14,15, 16],
    [2, 10, 0, 1],
    [11, 12, 8, 9],
    [3, 17,18, 19]
]


In [ ]:
def encode_labels(labels: List[List[int]]) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor
        Add labels for grouped classes
    Args:
        labels (List[List[int]]): List of all labels in the batch

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    
    extended_labels = []
    for i in range(len(labels)):
      elabels = [p for p in labels[i]]
      #Cluster 0
      for j in range(len(clusters)):
        clf = clusters[j]
        l = 0
        for k in clusters[j]:
          if(elabels[k]==1):
            l = 1
            break
        elabels.append(l)
      extended_labels.append(elabels)
    #print(len(extended_labels[0]), len(labels[0]))
    return torch.LongTensor(extended_labels)


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
train_label_batches = [b for b in chunk(train_labels, batch_size)]
train_label_batches = [encode_labels(batch) for batch in train_label_batches]

In [ ]:
val_label_batches = [b for b in chunk(val_labels, batch_size)]
val_label_batches = [encode_labels(batch) for batch in val_label_batches]

# **Grouped Classifier**

In [ ]:
class GroupedClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        # Updating all BERT parameters can be slow and memory intensive. 
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        for param in self.bert.parameters():
            param.requires_grad = True
        self.bert_hidden_dimension = self.bert.config.hidden_size
        print(self.bert_hidden_dimension)
        
        # Grouped layers
        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, 512)
        self.group_layer1 = torch.nn.Linear(512, 64)
        self.group_layer2 = torch.nn.Linear(512, 64)
        self.group_layer3 = torch.nn.Linear(512, 64)
        self.group_layer4 = torch.nn.Linear(512, 64)
        self.group_layer5 = torch.nn.Linear(512, 64)
        self.classifier_group1 = torch.nn.Linear(64, 1)
        self.classifier_group2 = torch.nn.Linear(64, 1)
        self.classifier_group3 = torch.nn.Linear(64, 1)
        self.classifier_group4 = torch.nn.Linear(64, 1)
        self.classifier_group5 = torch.nn.Linear(64, 1)
        
        # Seperate outputs for all the 20 classes
        self.hidden_layer1 = torch.nn.Linear(64, 32)
        self.hidden_layer2 = torch.nn.Linear(64, 32)
        self.hidden_layer3 = torch.nn.Linear(64, 32)
        self.hidden_layer4 = torch.nn.Linear(64, 32)
        self.hidden_layer5 = torch.nn.Linear(64, 32)
        self.hidden_layer6 = torch.nn.Linear(64, 32)
        self.hidden_layer7 = torch.nn.Linear(64, 32)
        self.hidden_layer8 = torch.nn.Linear(64, 32)
        self.hidden_layer9 = torch.nn.Linear(64, 32)
        self.hidden_layer10 = torch.nn.Linear(64, 32)
        self.hidden_layer11 = torch.nn.Linear(64, 32)
        self.hidden_layer12 = torch.nn.Linear(64, 32)
        self.hidden_layer13 = torch.nn.Linear(64, 32)
        self.hidden_layer14 = torch.nn.Linear(64, 32)
        self.hidden_layer15 = torch.nn.Linear(64, 32)
        self.hidden_layer16 = torch.nn.Linear(64, 32)
        self.hidden_layer17 = torch.nn.Linear(64, 32)
        self.hidden_layer18 = torch.nn.Linear(64, 32)
        self.hidden_layer19 = torch.nn.Linear(64, 32)
        self.hidden_layer20 = torch.nn.Linear(64, 32)
        self.relu = torch.nn.ReLU()
        self.classifier1 = torch.nn.Linear(32, 1)
        self.classifier2 = torch.nn.Linear(32, 1)
        self.classifier3 = torch.nn.Linear(32, 1)
        self.classifier4 = torch.nn.Linear(32, 1)
        self.classifier5 = torch.nn.Linear(32, 1)
        self.classifier6 = torch.nn.Linear(32, 1)
        self.classifier7 = torch.nn.Linear(32, 1)
        self.classifier8 = torch.nn.Linear(32, 1)
        self.classifier9 = torch.nn.Linear(32, 1)
        self.classifier10 = torch.nn.Linear(32, 1)
        self.classifier11 = torch.nn.Linear(32, 1)
        self.classifier12 = torch.nn.Linear(32, 1)
        self.classifier13 = torch.nn.Linear(32, 1)
        self.classifier14 = torch.nn.Linear(32, 1)
        self.classifier15 = torch.nn.Linear(32, 1)
        self.classifier16 = torch.nn.Linear(32, 1)
        self.classifier17 = torch.nn.Linear(32, 1)
        self.classifier18 = torch.nn.Linear(32, 1)
        self.classifier19 = torch.nn.Linear(32, 1)
        self.classifier20 = torch.nn.Linear(32, 1)
        
        self.log_softmax = torch.nn.LogSoftmax(dim=2)

    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        embedded = self.bert(**symbols)
        # Get the [CLS] token using the `pooler_output` from 
        #      The BertModel output. See here: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        #      and check the returns for the forward method.
        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension
        #raise NotImplementedError
        
        last_hidden_state = embedded.last_hidden_state[:,0,:]
        hidden_shape = last_hidden_state.shape
        return torch.reshape(last_hidden_state,(hidden_shape[0],1,hidden_shape[1]) )

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        #Grouping
        #Group1 - {'Achievement', 'Face', 'Power: dominance', 'Power: resources'}
        enc = self.hidden_layer(encoded_sents)
        group1 = self.group_layer1(enc)
        group1 = self.relu(group1)
        group_output1 = self.classifier_group1(group1)
        group_output1 = torch.nn.Sigmoid()(group_output1)
        
        #Group2 - {'Benevolence: caring', 'Benevolence: dependability', 'Humility', 'Universalism: concern'}
        group2 = self.group_layer2(enc)
        group2 = self.relu(group2)
        group_output2 = self.classifier_group2(group2)
        group_output2 = torch.nn.Sigmoid()(group_output2)
        
        #Group3 - {'Stimulation', 'Tradition', 'Self-direction: action', 'Self-direction: thought'}
        group3 = self.group_layer3(enc)
        group3 = self.relu(group3)
        group_output3 = self.classifier_group3(group3)
        group_output3 = torch.nn.Sigmoid()(group_output3)
        
        #Group4 - {'Conformity: interpersonal', 'Conformity: rules', 'Security: personal', 'Security: societal'}
        group4 = self.group_layer4(enc)
        group4 = self.relu(group4)
        group_output4 = self.classifier_group4(group4)
        group_output4 = torch.nn.Sigmoid()(group_output4)
        
        #Group5 - {'Hedonism, Universalism: nature', 'Universalism: objectivity', 'Universalism: tolerance'}
        group5 = self.group_layer5(enc)
        group5 = self.relu(group5)
        group_output5 = self.classifier_group5(group5)
        group_output5 = torch.nn.Sigmoid()(group_output5)
        
        output1 = self.hidden_layer1(group1)
        output1 = self.relu(output1)
        output1 = self.classifier1(output1)
        output1 = torch.nn.Sigmoid()(output1)
        
        output2 = self.hidden_layer2(group1)
        output2 = self.relu(output2)
        output2 = self.classifier2(output2)
        output2 = torch.nn.Sigmoid()(output2)
        
        output3 = self.hidden_layer3(group1)
        output3 = self.relu(output3)
        output3 = self.classifier3(output3)
        output3 = torch.nn.Sigmoid()(output3)
        
        output4 = self.hidden_layer4(group5)
        output4 = self.relu(output4)
        output4 = self.classifier4(output4)
        output4 = torch.nn.Sigmoid()(output4)
        
        output5 = self.hidden_layer5(group2)
        output5 = self.relu(output5)
        output5 = self.classifier5(output5)
        output5 = torch.nn.Sigmoid()(output5)
        
        
        output6 = self.hidden_layer6(group2)
        output6 = self.relu(output6)
        output6 = self.classifier6(output6)
        output6 = torch.nn.Sigmoid()(output6)
        
        
        output7 = self.hidden_layer7(group2)
        output7 = self.relu(output7)
        output7 = self.classifier7(output7)
        output7 = torch.nn.Sigmoid()(output7)
        
        
        output8 = self.hidden_layer8(group2)
        output8 = self.relu(output8)
        output8 = self.classifier8(output8)
        output8 = torch.nn.Sigmoid()(output8)
        
        output9 = self.hidden_layer9(group3)
        output9 = self.relu(output9)
        output9 = self.classifier9(output9)
        output9 = torch.nn.Sigmoid()(output9)
        
        output10 = self.hidden_layer10(group3)
        output10 = self.relu(output10)
        output10 = self.classifier10(output10)
        output10 = torch.nn.Sigmoid()(output10)
        
        output11 = self.hidden_layer11(group1)
        output11 = self.relu(output11)
        output11 = self.classifier11(output11)
        output11 = torch.nn.Sigmoid()(output11)
        
        output12 = self.hidden_layer12(group3)
        output12 = self.relu(output12)
        output12 = self.classifier12(output12)
        output12 = torch.nn.Sigmoid()(output12)
        
        output13 = self.hidden_layer13(group3)
        output13 = self.relu(output13)
        output13 = self.classifier13(output13)
        output13 = torch.nn.Sigmoid()(output13)
        
        output14 = self.hidden_layer14(group4)
        output14 = self.relu(output14)
        output14 = self.classifier14(output14)
        output14 = torch.nn.Sigmoid()(output14)
        
        output15 = self.hidden_layer15(group4)
        output15 = self.relu(output15)
        output15 = self.classifier15(output15)
        output15 = torch.nn.Sigmoid()(output15)
        
        output16 = self.hidden_layer16(group4)
        output16 = self.relu(output16)
        output16 = self.classifier16(output16)
        output16 = torch.nn.Sigmoid()(output16)
        
        output17 = self.hidden_layer17(group4)
        output17 = self.relu(output17)
        output17 = self.classifier17(output17)
        output17 = torch.nn.Sigmoid()(output17)
        
        output18 = self.hidden_layer18(group5)
        output18 = self.relu(output18)
        output18 = self.classifier18(output18)
        output18 = torch.nn.Sigmoid()(output18)
        
        output19 = self.hidden_layer19(group5)
        output19 = self.relu(output19)
        output19 = self.classifier19(output19)
        output19 = torch.nn.Sigmoid()(output19)
        
        output20 = self.hidden_layer20(group5)
        output20 = self.relu(output20)
        output20 = self.classifier20(output20)
        output20 = torch.nn.Sigmoid()(output20)
        outputs = torch.cat((output1, output2, output3, output4, output5, output6, output7, output8, output9, output10, output11, output12, output13, output14, output15, output16, output17, output18, output19, output20),2)
        group_outputs = torch.cat((group_output1, group_output2, group_output3, group_output4, group_output5),2)
        return outputs, group_outputs

In [ ]:
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    sents = sents.to(device)
    logits = model(sents)[0]
    res = []
    logitslen = len(logits)
    #print(logits[0].shape)
    for i in range(logitslen):
        datares = []
        for j in range(20):
            datares.append(logits[i][0][j] > 0.5)
        res.append(datares)
    return res


## Evaluation

In [ ]:
def f1Score_multiLabel(preds, labels):
    nLabels = 20
    relevants = [0]*20
    positives = [0]*20
    truePositives = [0]*20
    correct = [0]*20
    for i in range(len(preds)):
        for j in range(nLabels):
            if(preds[i][j]==1):
                positives[j] += 1
                if(labels[i][j]==1):
                    truePositives[j] += 1
    for i in range(len(preds)):
        for j in range(nLabels):
            if(preds[i][j]==labels[i][j]):
                correct[j] += 1
    
    for i in range(len(labels)):
        for j in range(nLabels):
            if(labels[i][j]==1):
                relevants[j] += 1
    
    precisions = []*nLabels
    recalls = []*nLabels
    f1Scores = []*nLabels
    accuracies = []*nLabels
    
    #print(truePositives, positives, relevants)
    for i in range(nLabels):
        precision =0
        recall = 0
        f1 = 0    
        if(positives[i]>0):
            precision = truePositives[i]/positives[i]
        precisions.append(precision)
        if(relevants[i]>0):
            recall = truePositives[i]/relevants[i]
        recalls.append(recall)
        #print(precision,recall,i)
        if(precision>0 and recall>0):
            f1 = 2 * precision * recall / (precision + recall)
        f1Scores.append(f1)
        accuracies.append(correct[i]/len(preds))
    precision_mean = np.mean(precisions)
    recall_mean = np.mean(recalls)
    f1_mean = np.mean(f1Scores)
    accuracy = np.mean(accuracies)
    return f1_mean, precision_mean, recall_mean, accuracy, f1Scores, precisions, recalls, accuracies
    


# Training

In [ ]:
import random
from tqdm import tqdm_notebook as tqdm
from torchmetrics.classification import MultilabelHammingDistance
def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
    best_f1,
):
    print("Training...")
    all_f1 = []
    all_P = []
    all_R = []
    all_L = []
    all_CELoss = []
    all_HMLoss = []
    all_acc = []
    loss_func = torch.nn.CrossEntropyLoss()
    hammingLoss20 = MultilabelHammingDistance(num_labels=20)
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        for features, labels in tqdm(batches):
            # Empty the dynamic computation graph
            features = features.to(device)
            labels = labels.float()
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(features)
            local_loss = loss_func(preds[0].squeeze(1),labels[:,:20])
            group_loss = loss_func(preds[1].squeeze(1),labels[:,20:25])
            loss = local_loss + 4*group_loss
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {np.sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            sents = sents.to(device)
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels))
        # #print(range(len(set(train_labels))))

        dev_f1, dev_P, dev_R, dev_acc, dev_all_f1, dev_all_P, dev_all_R, dev_all_acc = f1Score_multiLabel(all_preds, all_labels)
        print(f"Dev F1 {dev_f1},  Dev Precision {dev_P}, Dev Recall {dev_R}, Dev Accuracy {dev_acc}")
        print(dev_all_P[2:4],dev_all_P[12:14])
        if dev_f1 > best_f1:
            torch.save(model, 'best-f1.pt')
            best_f1 = dev_f1
            print("Best F1 ", best_f1)
        all_f1.append(dev_all_f1)
        all_P.append(dev_all_P)
        all_R.append(dev_all_R)
        all_L.append(losses)
        all_acc.append(dev_all_acc)
       
    # Return the trained model
    with open("all_f1_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_f1,
               delimiter =", ", 
               fmt ='%s')
    with open("all_P_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_P,
               delimiter =", ", 
               fmt ='%s')
    with open("all_R_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_R,
               delimiter =", ", 
               fmt ='%s')
    with open("all_acc_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_acc,
               delimiter =", ", 
               fmt ='%s')
    with open("all_L_base.csv", 'ab') as abc:
        np.savetxt(abc, 
               all_L,
               delimiter =", ", 
               fmt ='%s')
    
    return model

In [ ]:
from transformers.optimization import get_linear_schedule_with_warmup
epochs = 300
epoch_warmup = 40
best_f1 = 0
# TODO: Find a good learning rate
LR = 1e-4

possible_labels = 20
model = GroupedClassifier(output_size=possible_labels, hidden_size=512)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), LR)
#scheduler = get_linear_schedule_with_warmup(optimizer, epoch_warmup,epochs)


In [ ]:
#epochs=50
#LR=1e-4
optimizer = torch.optim.AdamW(model.parameters(), LR)
model =training_loop(
    epochs,
    train_input_batches,
    train_label_batches,
    val_input_batches,
    val_label_batches,
    optimizer,
    model,
    best_f1,
)

In [ ]:
model = torch.load('best-f1.pt')

Validation Set

In [ ]:
print("Evaluating dev...")
all_preds_val = []
all_labels = []
for sents, labels in tqdm(zip(val_input_batches, val_label_batches), total=len(val_input_batches)):
    sents = sents.to(device)
    pred = predict(model, sents)
    all_preds_val.extend(pred)
    all_labels.extend(list(labels))
# #print(range(len(set(train_labels))))

dev_f1, dev_P, dev_R, dev_acc, dev_all_f1, dev_all_P, dev_all_R, dev_all_acc = f1Score_multiLabel(all_preds_val, all_labels)
print(f"Dev F1 {dev_f1},  Dev Precision {dev_P}, Dev Recall {dev_R}, Dev Accuracy {dev_acc}")


In [ ]:
all_preds1_val = []
for row in all_preds_val:
    row1 = []
    for item in row:
        val = item.cpu().numpy()
        if val == True:
            row1.append(1)
        else:
            row1.append(0)
    all_preds1_val.append(row1)

In [ ]:
label_ids = ['Self-direction: thought', 'Self-direction: action', 'Stimulation',
       'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources',
       'Face', 'Security: personal', 'Security: societal', 'Tradition',
       'Conformity: rules', 'Conformity: interpersonal', 'Humility',
       'Benevolence: caring', 'Benevolence: dependability',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: tolerance', 'Universalism: objectivity']

In [ ]:
all_preds_df_val = pd.DataFrame(all_preds1_val, columns=label_ids)
write_tsv_dataframe('preds-val-GroupingHybrid.tsv', all_preds_df_val)

## Test Set 

In [ ]:
def generate_inputs(dataset, datatype):
    """
    TODO: group all premises and corresponding hypotheses and labels of the datapoints
    a datapoint as seen earlier is a dict of premis, hypothesis and label
    """
    #raise NotImplementedError
    premise=[]
    conclusion=[]
    stance=[]
    
    label=[]
    
    n = len(dataset['Argument ID'])
    arguments = []
    print(n)
    for i in range(n):
        if dataset['Usage'][i]==datatype:
          premise.append(dataset['Premise'][i])
          conclusion.append(dataset['Conclusion'][i])
          stance.append(dataset['Stance'][i])
          arguments.append(dataset['Argument ID'][i])
    
    return premise, conclusion, stance, arguments

In [ ]:
test_argument_filepath = os.path.join(data_dir, 'arguments-test.tsv')
df_arguments_test = load_arguments_from_tsv(test_argument_filepath, default_usage='test')
test_premises, test_conclusion, test_stance, test_arguments = generate_inputs(df_arguments_test, 'test')

In [ ]:
print("Evaluating test...")
all_preds = []
all_labels = []
batch_size=64
tokenizer = BatchTokenizer()

test_input_batches = [b for b in chunk_multi(test_premises, test_conclusion, test_stance, batch_size)]
# Tokenize + encode
test_input_batches = [tokenizer(*batch) for batch in test_input_batches]


#test_label_batches = [b for b in chunk(test_labels, batch_size)]
#test_label_batches = [encode_labels(batch) for batch in test_label_batches]
for sents in tqdm(test_input_batches):
    pred = predict(model, sents)
    all_preds.extend(pred)

In [ ]:
all_preds1 = []
for row in all_preds:
    row1 = []
    for item in row:
        val = item.cpu().numpy()
        if val == True:
            row1.append(1)
        else:
            row1.append(0)
    all_preds1.append(row1)

In [ ]:
label_ids = ['Self-direction: thought', 'Self-direction: action', 'Stimulation',
       'Hedonism', 'Achievement', 'Power: dominance', 'Power: resources',
       'Face', 'Security: personal', 'Security: societal', 'Tradition',
       'Conformity: rules', 'Conformity: interpersonal', 'Humility',
       'Benevolence: caring', 'Benevolence: dependability',
       'Universalism: concern', 'Universalism: nature',
       'Universalism: tolerance', 'Universalism: objectivity']

In [ ]:
all_preds_df = pd.DataFrame(all_preds1, columns=label_ids)

In [ ]:
all_preds_df

In [ ]:
write_tsv_dataframe('preds-test-GroupingHybrid.tsv', all_preds_df)